## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here

players = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')

In [4]:
players

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,...,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,...,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,...,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [10]:
# answer goes here

players_df = players.loc[players['GS'] >= 20]
players_df = players.loc[players['MP'] >= 10]

players_df

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,2016,Tony Wroten,PG,22,PHI,8,3,18.0,2.8,8.1,...,0.541,0.3,2.4,2.6,2.5,0.4,0.0,3.6,1.4,8.4
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,...,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,...,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [13]:
# answer goes here

keep_cols = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']

X = players_df[keep_cols]

X

,PTS,TRB,AST,STL,BLK,TOV
0,5.3,1.5,0.6,0.5,0.2,0.5
1,1.7,2.5,0.8,0.1,0.4,0.4
2,3.2,1.8,1.9,0.4,0.1,0.8
3,13.9,9.5,1.6,1.5,1.0,1.7
4,8.9,7.3,2.2,0.9,0.8,1.5
...,...,...,...,...,...,...
2134,8.4,2.6,2.5,0.4,0.0,3.6
2137,7.3,1.8,0.6,0.4,0.1,0.6
2138,15.1,9.0,1.8,1.5,0.5,1.9
2139,8.7,6.2,1.0,0.8,0.9,0.9


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [14]:
# answer goes here

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_std = scaler.fit_transform(X)

Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [15]:
# answer goes here

from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=3)

k_means.fit(X_std)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [19]:
k_means.cluster_centers_

array([[-0.51747093, -0.49480871, -0.3645032 , -0.40455108, -0.3644941 ,
        -0.50025289],
       [ 1.42821269,  0.26613422,  1.73120672,  1.26775197, -0.07499937,
         1.62897124],
       [ 0.50539725,  1.31770244, -0.19434568,  0.28036838,  1.17636359,
         0.30016017]])

In [22]:
pd.DataFrame(scaler.inverse_transform(k_means.cluster_centers_), columns=X.columns)

,PTS,TRB,AST,STL,BLK,TOV
0,6.746566,2.842284,1.485638,0.549688,0.283854,0.872435
1,17.696390,4.659928,5.181949,1.227076,0.413718,2.505415
2,12.503014,7.171781,1.785753,0.827123,0.975068,1.486301


I notice that one group has significantly high points and assists, one cluster has high rebounds, and the last is kind of overall poor performers.

Experiment with different values of K. Do any further interesting archetypes come out?

In [23]:
# answer goes here

k_means = KMeans(n_clusters=4)
k_means.fit(X_std)
pd.DataFrame(scaler.inverse_transform(k_means.cluster_centers_), columns=X.columns)

,PTS,TRB,AST,STL,BLK,TOV
0,19.837952,5.399398,6.106627,1.374699,0.493976,2.871687
1,5.765362,2.778648,1.107592,0.444721,0.298695,0.733926
2,11.272849,3.783556,2.659847,0.922371,0.330019,1.442639
3,12.897835,8.126407,1.792641,0.746753,1.222078,1.578355


In [24]:
k_means = KMeans(n_clusters=5)
k_means.fit(X_std)
pd.DataFrame(scaler.inverse_transform(k_means.cluster_centers_), columns=X.columns)

,PTS,TRB,AST,STL,BLK,TOV
0,5.648912,2.669142,1.072983,0.436492,0.275160,0.713316
1,19.476364,5.117576,6.026061,1.364848,0.447879,2.796970
2,17.607500,10.187500,2.463750,0.941250,1.752500,2.193750
3,10.563376,6.347134,1.550000,0.743312,0.797134,1.269745
4,11.057920,3.209693,2.923168,0.905910,0.258629,1.469740


In [25]:
k_means = KMeans(n_clusters=2)
k_means.fit(X_std)
pd.DataFrame(scaler.inverse_transform(k_means.cluster_centers_), columns=X.columns)

,PTS,TRB,AST,STL,BLK,TOV
0,15.961243,6.004570,3.655941,1.070932,0.679342,2.091773
1,6.823684,3.133388,1.441447,0.552796,0.343010,0.880181


With k=2; clusters are high performers and low performers